## Lets start from eugenesiow/bart-paraphrase model

In [1]:
import warnings

from datasets import load_dataset, load_metric
import transformers
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

warnings.filterwarnings('ignore')

In [2]:
# selecting model checkpoint
model_checkpoint = "eugenesiow/bart-paraphrase"

In [3]:
# setting random seed for transformers library
transformers.set_seed(42)

# Load the BLUE metric
metric = load_metric("sacrebleu")

### Dataset

In [4]:
train_df = pd.read_csv('../data/internal/train.csv')
test_df = pd.read_csv('../data/internal/test.csv')
val_df = pd.read_csv('../data/internal/validation.csv')

In [5]:
from transformers import BartForConditionalGeneration, BartTokenizer

model = BartForConditionalGeneration.from_pretrained(model_checkpoint)

In [6]:
tokenizer = BartTokenizer.from_pretrained('eugenesiow/bart-paraphrase')

In [7]:
prefix = "detoxify:"
source = "reference"
target = "translation"
max_input_length = 128
max_target_length = 128
batch_size = 32

def preprocess_function(examples):
    inputs = [prefix + ex for ex in examples[source]]
    targets = [ex for ex in examples[target]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [18]:
from datasets import Dataset, DatasetDict

tds = Dataset.from_pandas(train_df)
vds = Dataset.from_pandas(val_df)
ds = DatasetDict()
ds['train'] = tds
ds['validation'] = vds

In [19]:
ds = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/346666 [00:00<?, ? examples/s]

Map:   0%|          | 0/115555 [00:00<?, ? examples/s]

In [20]:
ds['train']

Dataset({
    features: ['Unnamed: 0', 'id', 'reference', 'translation', 'similarity', 'lenght_diff', 'ref_tox', 'trn_tox', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 346666
})

In [22]:
ds_train_pieces = []
ds_val_pieces = []
n = 5000
dn = 5
for i in range(0,n,dn):
    ds_train_pieces.append(ds['train'].select(range(i, i+dn)))
    ds_val_pieces.append(ds['validation'].select(range(i, i+dn)))

# Fine-tuning

In [11]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source}-to-{target}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    report_to='tensorboard',
)

In [12]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [13]:
import numpy as np

# simple postprocessing for text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

# compute metrics function to pass to trainer
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [14]:
# instead of writing train loop we will use Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=ds['train'],
    eval_dataset=ds['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [24]:
def training_small_pieces():
    for i in range(len(ds_train_pieces)):
        trainer = Seq2SeqTrainer(
            model,
            args,
            train_dataset=ds_train_pieces[i],
            eval_dataset=ds_val_pieces[i],
            data_collator=data_collator,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics
        )
        trainer.train()
        print(f'{i} out of {len(ds_train_pieces)} trained')

In [25]:
training_small_pieces()

Epoch,Training Loss,Validation Loss


0 out of 1000 trained


Epoch,Training Loss,Validation Loss


1 out of 1000 trained


Epoch,Training Loss,Validation Loss


...

Epoch,Training Loss,Validation Loss


560 out of 1000 trained


Epoch,Training Loss,Validation Loss


561 out of 1000 trained


RuntimeError: MPS backend out of memory (MPS allocated: 9.92 GB, other allocations: 8.05 GB, max allowed: 18.13 GB). Tried to allocate 196.35 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [26]:
import torch
torch.save(model.state_dict(), '../models/pre_trained_4.pt')